<a href="https://colab.research.google.com/github/AshwinKotgire/Play_with_LLMs/blob/main/Prompt_Engineering/LLM_UI_and_Cust_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U trl transformers accelerate
!pip install -q datasets bitsandbytes einops wandb
!pip install langchain
!pip install sentence_transformers
# !pip install faiss-cpu
!pip install faiss-gpu
!pip install pypdf
!pip install  tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install hugchat

In [ ]:
from Cust_chain import Cust_Chain_obj

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import pipeline
import transformers
import torch

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import PDFMinerPDFasHTMLLoader
from langchain.document_loaders import PDFPlumberLoader
from langchain.text_splitter import TokenTextSplitter

from IPython.display import HTML

from langchain.document_loaders import OnlinePDFLoader


from langchain.text_splitter import SentenceTransformersTokenTextSplitter

from Cust_chain import Cust_Chain_obj

In [ ]:

model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
sys_prompt="""<<SYS>>You are an honest assistant , who gives factually correct answers while refering to the context. If the context does not have an answer or if context is not available you try your best to answer the question on your own.<</SYS>>"""

In [ ]:
chain=Cust_Chain_obj(model,tokenizer,None,sys_prompt)

In [ ]:
url="https://arxiv.org/pdf/1706.03762.pdf"

chain.load_pdf_doc(url)
chain.populate_vector_store()


In [ ]:
chain.FAISS_obj.similarity_search('What are vectors?',k=2)

[Document(page_content='##ing, combined with fact that the output embeddings are offset by one position, ensures that the predictions for position ican depend only on the known outputs at positions less than i. 3. 2 attention an attention function can be described as mapping a query and a set of key - value pairs to an output, where the query, keys, values, and output are all vectors. the output is computed as a weighted sum 3', metadata={'source': '/tmp/tmphy46x2_2/tmp.pdf', 'page': 2}),
 Document(page_content='##duction model relying entirely on self - attention to compute representations of its input and output without using sequence - aligned rnns or convolution. in the following sections, we will describe the transformer, motivate self - attention and discuss its advantages over models such as [ 17, 18 ] and [ 9 ]. 3 model architecture most competitive neural sequence transduction models have an encoder - decoder structure [ 5, 2, 35 ]. here, the encoder maps an input sequence of 

In [ ]:
from IPython.display import Markdown

In [ ]:
Markdown(chain.run('What are transformers , explain in 100 words?',2,True)[1])

cuda


Transformers are a type of neural network architecture introduced in the paper "Attention is All You Need" by Vaswani et al. in 2017. They have become a popular choice for natural language processing tasks such as machine translation, text generation, and question answering. The transformer model relies on self-attention mechanisms that allow it to attend to different parts of the input sequence simultaneously and weigh their importance. This allows the model to capture long-range dependencies and contextual relationships in the input data, leading to better performance on complex tasks.

In [ ]:
class Chain(Cust_Chain_obj):
  def __init__(self,model,tokenizer,FAISS_obj,sys_prompt,url,embeddings = HuggingFaceEmbeddings(),
               text_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=50)):
    super.init(model,tokenizer,FAISS_obj,sys_prompt)
    self.url=url
    self.load_pdf(self.url)
    self.populate_vector_store()
    def load_new_doc_and_vector_store(self,url):
      self.url=url
      self.load_pdf
      self.popuolate_vector_store()
      return 'Successful'

In [ ]:
%%writefile A/main.py
print('Hello from A/main')
import numpy as np
import pandas as pd


Writing A/main.py
